In [3]:
import pandas as pd
matches = pd.read_csv("web_scraping/matches.csv", index_col=0)
matches.dtypes
# create features to predict on
matches["date"] = pd.to_datetime(matches["date"])
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek
# create label to predict
matches["target"] = (matches["result"] == "W").astype("float")

data_df = matches[["venue_code","opp_code","hour","day_code","target"]]

In [ ]:
from FootballMatchDataset import FootballMatchDataset

dataset = FootballMatchDataset(data_df)

dataset[0]

In [4]:
from FootballMatchDataset import FootballMatchDataset
from FootballMatchClassfier import FootballMatchClassfier
from torch.utils.data import DataLoader, random_split
import torch 
from torch.utils.tensorboard import SummaryWriter

dataset = FootballMatchDataset(data_df)
train_dataset,test_dataset,val_dataset = random_split(dataset, [0.7,0.2,0.1])

trainloader = DataLoader(train_dataset,batch_size=64,shuffle=False)
valloader = DataLoader(val_dataset,batch_size=64,shuffle=False)
testloader = DataLoader(test_dataset,batch_size=64,shuffle=False)

model = FootballMatchClassfier(4)
optimiser = torch.optim.SGD(model.parameters(), lr=0.0001)
loss_function = torch.nn.BCELoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

epochs = 500
for i in range(epochs):    
    print(f'Epoch: {i}')
    writer = SummaryWriter()
    batch_id = 0     
    # Set the model to run on the device
    model = model.to(device)
    model.train(True)     
    for batch in trainloader:
        # get features and labels from the batch
        features,labels = batch
        features = features.to(device)
        labels = labels.to(device, non_blocking=True)
        # loss.backward does not overwrite, it adds. To stop this, we set the gradients back to zero. sets the .grad of all optimized tensors to zero
        optimiser.zero_grad()
        # make a prediction
        prediction = model(features)
        # calculate loss
        criterion = loss_function(prediction,labels.unsqueeze(1))
        # backward function calculates the gradient of the current tensor w.r.t graph leaves
        criterion.backward()
        # moves each parameter in the opposite direction of the gradient, proportional to the learning rate
        optimiser.step()
        writer.add_scalar('Loss', criterion.item(), batch_id)
        batch_id += 1
        # print(f'Batch: {batch_id}')

Epoch: 0


RuntimeError: Found dtype Double but expected Float